# Part 2 - Exploratory Data Analytics

We are using the output from the first notebook. 


In [4]:
import os
import sys
from pathlib import Path
CWD = str(Path.cwd().parent)
sys.path.append(CWD)
from datetime import datetime, timedelta

import pandas as pd
import datatable as dt
from datatable import f, by
from common.utilities import decorators 
print(CWD)

/Users/jordancarson/Projects/JPM/data-engineering-nyc


We are going to be using both `pandas` and `datatable` for our EDA (exploratory data analysis). The below data has not been transformed from source and is in the same shape and size as the entire NYC OpenData MVCC table. We have parameters within pd.read_csv that will us to transform while reading.

## Looking and Finding DQ Issues

In [50]:
collision_df = pd.read_csv(
    os.path.join(CWD, '.data', 'output.csv'),
    # parse_dates={'crash_date_time': ["crash_date", "crash_time"]},
    infer_datetime_format=True,
    dtype={'zip_code': "str"}
)

In [37]:
collision_df.shape

(1830092, 30)

Creating our Date fields.

In [74]:
collision_df['crash_date'] = pd.to_datetime(collision_df['crash_date'], format="%Y-%m-%d")
collision_df['crash_time'] = collision_df['crash_time'].str[11:] # slicing to get only the time value back
collision_df['year'] = collision_df['crash_date'].dt.year
collision_df['month'] = collision_df['crash_date'].dt.month
collision_df['day'] = collision_df['crash_date'].dt.day
collision_df['hour'] = collision_df['crash_time'].str[:2]
collision_df['minutes'] = collision_df['crash_time'].str[3:5]
collision_df['seconds'] = collision_df['crash_time'].str[6:]

In [73]:
collision_df['crash_time'].head(1).str[11:].str[6:]
    # print(i[10:])/

0    00
Name: crash_time, dtype: object

We can see that we have 1,830,092 rows, with 29 columns. A [data dictionary](https://data.cityofnewyork.us/api/views/h9gi-nx95/files/bd7ab0b2-d48c-48c4-a0a5-590d31a3e120?download=true&filename=MVCollisionsDataDictionary_20190813_ERD.xlsx) is available which shows the metadata and table relationship between the Motor Vehicle - Crashes, Vehicles and Persons.

In [15]:
collision_df['crash_date_time'].min(), collision_df['crash_date_time'].describe()

('2012-07-01T00:00:00.000 2021-10-15 00:05:00',
 count                                         1830092
 unique                                         956712
 top       2015-01-18T00:00:00.000 2021-10-15 08:00:00
 freq                                               51
 Name: crash_date_time, dtype: object)

The Collision Data **begins** on ***July 1st 2012*** and series **ends** on ***October 15th 2021***. 

In [16]:
# Next lets find the missing data
print(collision_df.shape[0] - collision_df.count())

crash_date_time                        0
Unnamed: 0                             0
borough                           564983
zip_code                          565203
latitude                          214300
longitude                         214300
location                          214300
on_street_name                    373516
off_street_name                   656307
number_of_persons_injured             18
number_of_persons_killed              31
number_of_pedestrians_injured          0
number_of_pedestrians_killed           0
number_of_cyclist_injured              0
number_of_cyclist_killed               0
number_of_motorist_injured             0
number_of_motorist_killed              0
contributing_factor_vehicle_1       5247
contributing_factor_vehicle_2     264256
collision_id                           0
vehicle_type_code1                  9986
vehicle_type_code2                313450
contributing_factor_vehicle_3    1705024
vehicle_type_code_3              1708718
cross_street_nam

We can see that some fields have almost **all** missing values. 

There are 565,203 cases with no zip_code and 564,983 with no borough. Latitude and Longitude data has about 214,300 missing entries. Along with the location - location is a tuple of the lat/long data. We should be able to infer the zip-code and borough from the geolocation fields and or from additional sources.

In [17]:
collision_df.loc[:, ['latitude', 'longitude']].describe()

,latitude,longitude
count,1.615792e+06,1.615792e+06
mean,4.067283e+01,-7.383508e+01
std,1.447575e+00,2.823822e+00
min,0.000000e+00,-2.013600e+02
25%,4.066845e+01,-7.397563e+01
50%,4.072168e+01,-7.392805e+01
75%,4.076903e+01,-7.386668e+01
max,4.334444e+01,0.000000e+00


Theres something wrong about these two geolocation fields, the min of latitude is 0.0 - The location of (0.0, 0.0) is known as Null Island, this is where the Prime meridian and the equator intersect. This location is **not** in NYC, thus this is a data quality issue. In fact, if you open the Tableau Dashboard that is paired with this project you will notice an issue with how the zip-codes are plotting on the map. The reason why this is occuring is we are taking the **AVG()** of all data points within the lat and long vectors. If we have a value with 0.0, it will negatively impact the rest of the vector. 

Thus, we should replace this latitude and longitude data with NaN / Null, when it is equal to 0.0.

In [24]:
# Confirm first - Finding the number of issues and a few IDs for 0.0 Latitude
len(collision_df[collision_df['latitude'] == 0.0]['collision_id']), collision_df[collision_df['latitude'] == 0.0]['collision_id'].head()

(2038,
 785785    3421906
 785806    3421927
 813121    3449248
 816864    3452991
 816865    3452992
 Name: collision_id, dtype: int64)

There are about 2,038 records with a latitude of 0.0 - I expect the longitude data to be the same IDs. As we can see below, the location field is a dictionary of lat/long, and when the latitude is 0.0 the longitude is 0.0. This is a data quality issue.

In [30]:
# filtering on a given collision ID to find 
collision_df[collision_df['collision_id'].isin([3452992, 3452991]) ]['location']

816864    {'latitude': '0.0', 'longitude': '0.0'}
816865    {'latitude': '0.0', 'longitude': '0.0'}
Name: location, dtype: object

In [ ]:
def plot_top_causes(data):
    data['incidents'] = 

    d = d.groupby('reason').agg({'incidents': np.sum}).sort_values(by='incidents')
    d.plot(kind='barh', legend=False, figsize=(10,10))
    plt.title('2012-2017 Top Contributing Factors to Motor Vehicle Collisions\nRed = Top 15 Causes')
    plt.xlabel('Incidents')
    plt.ylabel('')
    plt.axhline(len(d)-20.5, color='#CC0000')

In [33]:
collision_df['crash_date_time'] = pd.to_datetime(collision_df['crash_date_time'])

ParserError: Unknown string format: 2012-07-01T00:00:00.000 2021-10-15 10:40:00

# Adding Features

If we want to find the answers to some of Michael's questions we first need to include additional features in our result-set. 

In [ ]:
collision_df['year'] = collision_df['']

# Graphing

In [31]:
%matplotlib inline
def plot_distributions(data):
    
    ds = data[[c for c in data.columns if c not in ['day','year','month','dow','date','TMAX','TMIN','SNWD','PRCP','IsWeekend']]]
    c = 3
    r = math.ceil(len(ds.columns) / c)
    
    sns.set_style('white')
    plt.figure(figsize=(15,25))
    for i in range(0, len(ds.columns)):
        ax = plt.subplot(r, c, i+1)
        d = ds.iloc[:, i].dropna()
        m = int(d.max())

        bins = list(range(0, m, 1 if m<50 else 5))
        sns.distplot(d, ax=ax, bins=bins, hist_kws={'rwidth': 0.8})
        
    plt.subplots_adjust(hspace=0.5)
    plt.savefig('nypd_daily_records_dist.jpg', dpi=80)
    
plot_distributions(pvt_fixed)

NameError: name 'pvt_fixed' is not defined

In [8]:
data = dt.Frame(collision_df)

In [9]:
collision_df.head().T

,0,1,2,3,4
crash_date_time,2012-07-01T00:00:00.000 2021-10-15 10:40:00,2012-07-01T00:00:00.000 2021-10-15 12:18:00,2012-07-01T00:00:00.000 2021-10-15 15:00:00,2012-07-01T00:00:00.000 2021-10-15 18:00:00,2012-07-01T00:00:00.000 2021-10-15 19:30:00
Unnamed: 0,0,1,2,3,4
borough,MANHATTAN,MANHATTAN,NaN,MANHATTAN,MANHATTAN
zip_code,10013.0,10004.0,NaN,10007.0,10013.0
latitude,40.720854,40.704799,NaN,40.713692,40.719058
longitude,-74.003929,-74.016685,NaN,-74.013754,-74.012442
location,"{'latitude': '40.7208537', 'longitude': '-74.0...","{'latitude': '40.7047985', 'longitude': '-74.0...",NaN,"{'latitude': '40.7136919', 'longitude': '-74.0...","{'latitude': '40.719058', 'longitude': '-74.01..."
on_street_name,CANAL STREET,BATTERY PLACE,WATER STREET,WEST STREET,WEST STREET
off_street_name,WOOSTER STREET,WEST STREET,OLD SLIP,VESEY STREET,HARRISON STREET
number_of_persons_injured,0.0,0.0,0.0,0.0,0.0


1. Average number of accidents by month by borough
2. What are the most dangerous hours of the day for pedestrians?
3. Year with most accidents?
4. Visualize data
5. Any other insights we find, data quality